Poisson Equation in Parallel
===

NGSolve can be executed on a cluster using the MPI message passing interface.

For the MPI jupyter-tutorials we use the `ipyparallel` module. 


Start an ipycluster by executing 

`ipcluster start --engines=MPI -n 8`

in the console, for from this notebook: 
[start cluster](startcluster.ipynb)


In jupyter, we can then connect to the cluster via the `Client` object from ipyparallel. 

In [ ]:
from ipyparallel import Client
c = Client()
c.ids

We use mpi4py https://mpi4py.readthedocs.io/ for issuing MPI calls from Python. The %%px syntax magic causes parallel execution of that cell:

In [ ]:
%%px 
from mpi4py import MPI
comm = MPI.COMM_WORLD
print (comm.rank, comm.size)

Process 0 generates the mesh, and distributes it within the group of processes defined by the *communicator*. All other ranks receive a part of the mesh. The function mesh.GetNE(VOL) returns the local number of elements:

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square

if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.1).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))
print (mesh.GetNE(VOL))

We can define spaces, bilinear / linear forms, and gridfunctions in the same way as in sequential mode. But now, the degrees of freedom are distributed on the cluster following the distribution of the mesh. The finite element spaces define how the dofs match together.

In [ ]:
%%px
fes = H1(mesh, order=8, dirichlet=".*")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
pre = Preconditioner(a, "local")
# pre = Preconditioner(a, "bddc")

a.Assemble()

f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)

from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre.mat, printrates=comm.rank==0, maxiter=200, tol=1e-8)
gfu.vec.data = inv*f.vec

Parallel pickling allows to serialize the distributed solution and transfer it to the client. The process with rank=0 gets the whole mesh and computed solution, all other processes get the local parts of the mesh and solution:

In [ ]:
gfu = c[:]["gfu"]

We can now draw the whole solution using the the master process's `gfu[0]`.

In [ ]:
from ngsolve.webgui import Draw
Draw (gfu[0]);

Drawing `gfu[n]` will draw only part of the solution that the process with rank=`n` possesses. 

In [ ]:
Draw (gfu[3]);

We can also visualize the sub-domains obtained by the automatic partitioning, without using any computed solution, as follows.

In [ ]:
%%px
fesL2 = L2(mesh, order=0)
gfL2 = GridFunction(fesL2)
gfL2.vec[:] = comm.rank

In [ ]:
gfL2 = c[:]["gfL2"]
Draw (gfL2[0]);